## BSMLH

In [1]:
#!/usr/bin/env python

from __future__ import print_function
import sys    
import rospy
import actionlib
from ouass.msg import PlanningAction, PlanningGoal
from ouass.msg import Data
from ouass.msg import RobotTarget
from nav_msgs.msg import Odometry
from geometry_msgs.msg import PoseStamped
import time
import ipywidgets as widgets
from IPython.display import display, clear_output


# title
text

In [2]:
# Create text input widgets for user input
x_text = widgets.FloatText(description="X Coordinate:")
y_text = widgets.FloatText(description="Y Coordinate:")
buttonChange = widgets.Button(description="Change Target")
buttonCancel = widgets.Button(description="Cancel Target")
buttonSubmit = widgets.Button(description="Submit")


# title
text

In [3]:
def cancel(b):
    client.cancel_goal()
    rospy.loginfo("Goal has been canceled\n")
    clear_output()
    display(buttonChange, buttonCancel)
    print("Goal has been canceled\n")


# title
text

In [4]:
def change_target(b):
    clear_output()
    display(x_text, y_text, buttonSubmit)


# title
text

In [5]:
def on_submit_clicked(b):
    x = x_text.value
    y = y_text.value

    # Publishing the last target for reference
    last_target_msg = RobotTarget()
    last_target_msg.target_x = x
    last_target_msg.target_y = y
    
    pub2.publish(last_target_msg)
    
    # Wait for the action server
    client.wait_for_server()
    
    # Initializing the goal  
    goal = PoseStamped()
    goal.pose.position.x = x
    goal.pose.position.y = y
    
    # Setting the goal
    goal = PlanningGoal(goal)
    
    # Sending the goal to the action server    
    client.send_goal(goal)
    
    clear_output()
    display(buttonChange, buttonCancel)
    print(f"New goal has been set, x: {x}, y: {y}\n")


# title
text

In [6]:
def subscriber_callback(data):
    # Declaring the custom message
    msg = Data()
    
    # Getting the current positions and velocities
    msg.vel_x = data.twist.twist.linear.x 
    msg.vel_y = data.twist.twist.linear.y
    msg.position_x = data.pose.pose.position.x 
    msg.position_y = data.pose.pose.position.y 

    # Publishing the message
    pub.publish(msg)


# title
text

In [7]:
def main():
    clear_output()
    display(buttonChange, buttonCancel)


# title
text

In [8]:
# Node initialization
rospy.init_node('UI', anonymous=True)

        # Client initialization and setting up the server
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)

        # Subscribers
sub = rospy.Subscriber("/odom", Odometry, subscriber_callback)

        # Publishers
pub = rospy.Publisher("/posvelo", Data, queue_size=10)
pub2 = rospy.Publisher('last_target', RobotTarget, queue_size=10)

        # Widget button click event handlers
buttonChange.on_click(change_target)
buttonCancel.on_click(cancel)
buttonSubmit.on_click(on_submit_clicked)


main()


Button(description='Change Target', style=ButtonStyle())

Button(description='Cancel Target', style=ButtonStyle())

New goal has been set, x: 1.0, y: 2.0



# Robot and Target Positions
text

# title
text